In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore
import numpy as np

In [ ]:

scaler = MinMaxScaler()


base_folder = Path('./result/ud-pipe')
nilc_reference_complete = pd.read_csv(base_folder / 'reference-complete.csv')
nilc_reference_simplified = pd.read_csv(base_folder / "reference-simplified.csv")
nilc_generated = pd.read_csv(base_folder / "generated-simplified.csv")

In [ ]:
nilc_generated

In [ ]:
nilc_reference = pd.concat([nilc_reference_complete,nilc_reference_simplified])

In [ ]:
nilc_reference["generated_with"] = "Original"

In [ ]:
merged = pd.concat([nilc_reference,nilc_generated])

In [ ]:
proportional_metrics = [
 'personal_pronoun_ratio', # can also be a positive sign according to Unicamp,
]

inverse_proportional_metrics = [

    "coreference_pronoun_ratio",
    'demonstrative_pronoun_ratio',
    'non_svo_ratio',
    'passive_voice_ratio',
    'words_before_main_verb_mean', # use different scaling for this metric?
    "long_sentence_ratio",
    "foreign_word_ratio",
]

complete_metrics = proportional_metrics + inverse_proportional_metrics

In [ ]:
merged

### Teste T

In [ ]:
import numpy as np 
import scipy.stats as st
import csv
from typing import Any
import math 
confidence = 0.95
confidence_intervals: list[dict[str,Any]] = []




for metric in inverse_proportional_metrics:
    per_group = merged.groupby("generated_with")[metric]
    for row in per_group.agg(["mean","count",st.sem]).iterrows():
        range_min,range_max = st.t.interval(confidence=confidence,df=row[1]["count"]-1,loc=row[1]["mean"],scale=row[1]["sem"])
        confidence_intervals.append({"metric":metric,"model":row[0], "range_min":range_min,"mean": row[1]["mean"], "range_max":range_max})

df_inverse_proportional = pd.DataFrame(confidence_intervals)
df_inverse_proportional[["range_min","mean","range_max"]] = df_inverse_proportional[["range_min","mean","range_max"]].map(lambda x: math.trunc(10000*x) / 10000)
df_inverse_proportional.to_csv("confidence_intervals_nilc_inverse_proportional.csv",index=False)

confidence_intervals = []
for metric in proportional_metrics:
    per_group = merged.groupby("generated_with")[metric]
    for row in per_group.agg(["mean","count",st.sem]).iterrows():
        range_min,range_max = st.t.interval(confidence=confidence,df=row[1]["count"]-1,loc=row[1]["mean"],scale=row[1]["sem"])
        confidence_intervals.append({"metric":metric,"model":row[0], "range_min":range_min,"mean": row[1]["mean"], "range_max":range_max})

df_proportional = pd.DataFrame(confidence_intervals)
df_proportional[["range_min","mean","range_max"]] = df_proportional[["range_min","mean","range_max"]].map(lambda x: math.trunc(10000*x) / 10000)
df_proportional.to_csv("confidence_intervals_nilc_proportional.csv",index=False)
